In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_validate
from sklearn import metrics
#import data from files

x_header = pd.read_csv("x_header.csv")
train_data = pd.read_csv('train_data.csv', header = None)
test_data = pd.read_csv('test_data.csv', header = None)
train_labels = pd.read_csv('train_labels.csv', header = None, dtype = 'category' )


#create variables 
header = pd.DataFrame(data = x_header)
X = pd.DataFrame(data = train_data)
X.columns = header.ID
X_test = pd.DataFrame(data = test_data)
X_test.columns = header.ID
y = pd.DataFrame(data = train_labels)
y.columns = ['Genre']


In [2]:
#Merge X and X_test

X = pd.concat([X,X_test])

#View value frequencies from each columns 
#for i in X.columns:
#    print("Value count", i, ": ", X[i].value_counts().head(5))
#Drop bad data

X = X.drop(columns = ['MFCC_Mean_1','MFCC_Mean_2','MFCC_Mean_3','MFCC_Mean_4','MFCC_Min_1'])


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10907 entries, 0 to 6543
Columns: 259 entries, Rhythm_Mean_1 to MFCC_Max_12
dtypes: float64(259)
memory usage: 21.6 MB
None


In [3]:
#Normalize data according to group
group = ((0,24),(24,48),(48,72),(72,96),(96,120),(120,144),(144,168),(216,224),(224,236),(236,247),(247,259))

def normalization(X,group):
    for i,j in group:
        maximum = np.amax(np.amax(X.iloc[:,i:j]))
        minimum = np.amin(np.amin(X.iloc[:,i:j]))
        dfnorm = np.divide(np.subtract(X.iloc[:,i:j],minimum),(maximum - minimum))
        X.iloc[:,i:j] = dfnorm
    return X

X = normalization(X,group)
X_test = X

#display(HTML(x_y_train.head(15).to_html(max_rows=15)))


In [4]:
#Split X into X and X_test
X_test = X[4363:]
X = X[:4363]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4363 entries, 0 to 4362
Columns: 259 entries, Rhythm_Mean_1 to MFCC_Max_12
dtypes: float64(259)
memory usage: 8.7 MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6544 entries, 0 to 6543
Columns: 259 entries, Rhythm_Mean_1 to MFCC_Max_12
dtypes: float64(259)
memory usage: 13.0 MB
None


In [9]:
#Split data to test-validation set
def run_model(model, alg_name):
    global X
    global y
    y = np.ravel(y)
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4, random_state=42)
    model.fit(X_train, y_train)
    print("----",alg_name,"----")
    print(model.score(X_train,y_train))
    print(model.score(X_val,y_val))
    #Apply cross validation
    scores = cross_validate(model, X_train, y_train, cv=3)
    print("Cross-validated scores:", scores["test_score"])

#try this  https://www.kaggle.com/coolman/different-classification-techniques-python
#Neural network results highest score
# ----------- Neural network - Multi-layer Perceptron  ------------
from sklearn.neural_network import MLPClassifier

model2 = MLPClassifier(max_iter = 500,activation = 'logistic',alpha = 0.0005, batch_size = 200)
run_model(model2, " MLP Neural network default")


----  MLP Neural network default ----
0.670997325182
0.630011454754


/opt/conda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Cross-validated scores: [ 0.63242009  0.62614679  0.64556962]


/opt/conda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [6]:
y_test_pred = model2.predict(X_test)
indx = X_test.index
result = pd.DataFrame(index = indx, columns = ["Sample_id","Sample_label"])
result["Sample_id"] = result.index+1
result["Sample_label"] = y_test_pred
result.to_csv("MLPClassifier_Merge_solution.csv", index = False)
